In [1]:
# Import Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

#Import Other useful libraries
import time
import pandas as pd
from datetime import datetime
import csv
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import os
from urllib.parse import urljoin
import time
import re

In [2]:
df_cities = pd.read_csv("C:\\Users\\mosta\\Downloads\\csvData.csv")

In [3]:
city_names = df_cities.name.unique()

In [4]:
brokers_info = {}

brokers_info["names"] = []
brokers_info["addresses"] = []
brokers_info["phones"] = []
brokers_info["websites"] = []
brokers_info["cities"] = []
brokers_info["companies"] = []


driver = webdriver.Chrome("C:\\Program Files (x86)\\chromedriver.exe")
driver.get("https://findamortgagebroker.com/")
driver.maximize_window()


start = time.time()

for city in city_names[:10]:
    search = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.ID, "contact-search-field")))
    search.clear()
    search.send_keys(city)
    search.send_keys(Keys.RETURN)

    try:
        #get the results of page 1
        results = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.ID, "totalResults")))
        brokers_1 = WebDriverWait(driver, 100).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "clickable-tile-contact")))
        results_num = int("".join(re.findall("[0-9]", results.text)))
        pages_num = (results_num+1)//20
        for i in range(len(brokers_1)):
            brokers_1 = WebDriverWait(driver, 100).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "clickable-tile-contact")))
            brokers_1[i].click()
            name = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CLASS_NAME, "banner-primary-text")))
            info_div = driver.find_element(By.CLASS_NAME, "contact-info-list")
            infos = info_div.find_elements(By.TAG_NAME, "a")
            company_div = driver.find_element(By.CLASS_NAME, "company-info")
            try:
                company = company_div.find_element(By.TAG_NAME,"h6").text
            except:
                company = "None"
            try:
                address = infos[0].text
            except IndexError:
                address = "None"
            try:
                phone = infos[1].text
            except IndexError:
                phone = "None"
            try:
                website = infos[2].text
            except IndexError:
                website = "None"          
            brokers_info["names"].append(name.text)
            brokers_info["addresses"].append(address)
            brokers_info["phones"].append(phone)
            brokers_info["websites"].append(website)
            brokers_info["cities"].append(city)
            brokers_info["companies"].append(company)
            driver.back()
        #get the results of the other pages
        for n in range(2,pages_num+1):
            x = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "current-page")))
            drop = Select(x)
            drop.select_by_visible_text(f"{n}")
            time.sleep(2)
            brokers = WebDriverWait(driver, 100).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "clickable-tile-contact")))
            for i in range(len(brokers)):
                brokers = WebDriverWait(driver, 100).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "clickable-tile-contact")))  
                brokers[i].click()
                name = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CLASS_NAME, "banner-primary-text")))
                info_div = driver.find_element(By.CLASS_NAME, "contact-info-list")
                infos = info_div.find_elements(By.TAG_NAME, "a")
                company_div = driver.find_element(By.CLASS_NAME, "company-info")
                try:
                    company = company_div.find_element(By.TAG_NAME,"h6").text
                except:
                    company = "None"
                try:
                    address = infos[0].text
                except IndexError:
                    address = "None"
                try:
                    phone = infos[1].text
                except IndexError:
                    phone = "None"
                try:
                    website = infos[2].text
                except IndexError:
                    website = "None"               
                brokers_info["names"].append(name.text)
                brokers_info["addresses"].append(address)
                brokers_info["phones"].append(phone)
                brokers_info["websites"].append(website)
                brokers_info["cities"].append(city)
                brokers_info["companies"].append(company)
                driver.back()
    except:
        driver.quit()
    driver.back()
    time.sleep(1)
end = time.time()
duration = end - start

<ipython-input-4-d6c0e790c72a>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Program Files (x86)\\chromedriver.exe")


In [5]:
df = pd.DataFrame(brokers_info)

In [6]:
df.head()

,names,addresses,phones,websites,cities,companies
0,David Saraf,711 forest avenue staten island NY 10310,718-720-8600,https://www.tristarmortgage.com/,New York City,Tristar Mortgage Corp
1,Suresh Budhoo,105-10 Jamaica Avenue Richmond Hill NY 11418,718-845-4800,https://n/A,New York City,S.B. Funding Corp.
2,Sergey Radoff,2160 NORTH CENTRAL RD Suite 206A Fort Lee NJ 0...,800-353-5114 1,https://www.tristatemortgages.com/,New York City,Tri State Mortgage Corporation
3,Vadim Zaurov,3430 Sunset Ave. Suite 17 Ocean NJ 07712,7326881726,https://aafcgroup.com/,New York City,AAFC Group LLC
4,Johny Jack Rukab,711 Forest Ave STATEN ISLAND NY 10310,718-720-8600,https://www.zillow.com/lender-profile/johnyRukab/,New York City,Tristar Mortgage Corp


In [10]:
df.to_csv("brokers.csv")

In [11]:
df.cities.unique()

array(['New York City', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix',
       'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose'],
      dtype=object)